In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/crossselling/CatalogCrossSell.csv


In [2]:
%matplotlib inline

from pathlib import Path

import heapq
from collections import defaultdict

import matplotlib.pylab as plt
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split

In [3]:
cs = pd.read_csv('../input/crossselling/CatalogCrossSell.csv')

In [4]:
cs.head(11)

,Customer Number,Clothing Division,Housewares Division,Health Products Division,Automotive Division,Personal Electronics Division,Computers Division,Garden Division,Novelty Gift Division,Jewelry Division
0,11569,0,1,1,1,1,0,0,1,0
1,13714,0,1,1,1,1,0,1,1,1
2,46391,0,1,1,1,1,0,1,1,1
3,67264,0,0,1,1,1,0,1,1,0
4,67363,0,0,1,0,1,0,1,1,0
5,72553,0,1,1,1,1,0,1,1,1
6,79814,0,1,1,0,1,0,1,0,0
7,80903,0,1,1,0,1,0,0,1,0
8,91439,0,0,1,1,1,0,1,0,1
9,96701,0,1,1,1,1,0,1,1,1


In [5]:
cs.isnull().sum()

Customer Number                  0
Clothing Division                0
Housewares Division              0
Health Products Division         0
Automotive Division              0
Personal Electronics Division    0
Computers Division               0
Garden Division                  0
Novelty Gift Division            0
Jewelry Division                 0
dtype: int64

In [6]:
len(cs)

4998

In [7]:
cs.drop(['Customer Number'], axis = 1, inplace = True)

In [8]:
itemsets = apriori(cs, min_support=1000/4998, use_colnames=True)

# and convert into rules
rules = association_rules(itemsets, metric='confidence', min_threshold=0.5)
rules.sort_values(by=['lift'], ascending=True)

print(rules.sort_values(by=['lift'], ascending=True)
      .drop(columns=['antecedent support', 'consequent support', 'conviction', 'leverage']))

                                          antecedents  \
0                               (Housewares Division)   
3                     (Personal Electronics Division)   
4                                   (Garden Division)   
5                             (Novelty Gift Division)   
6                                  (Jewelry Division)   
9   (Personal Electronics Division, Housewares Div...   
1                     (Personal Electronics Division)   
2                               (Housewares Division)   
7   (Health Products Division, Personal Electronic...   
8     (Health Products Division, Housewares Division)   
10                    (Personal Electronics Division)   
11                              (Housewares Division)   

                                          consequents   support  confidence  \
0                          (Health Products Division)  0.393557    1.000000   
3                          (Health Products Division)  0.467387    1.000000   
4                    